In [3]:
#importing library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso

In [16]:
#importing the data

pd.set_option('display.max_columns', 500)
df=pd.read_csv('../input/spctornado/Tornadoes_SPC_1950to2015.csv')
#loading the data
df

,om,yr,mo,dy,date,time,tz,st,stf,stn,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,fc
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,0
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,0
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,607506,2015,12,28,12/28/2015,3:20:00,3,LA,22,0,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150,0
60110,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0
60111,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0
60112,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0


In [17]:
#getting informatio about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60114 entries, 0 to 60113
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   om      60114 non-null  int64  
 1   yr      60114 non-null  int64  
 2   mo      60114 non-null  int64  
 3   dy      60114 non-null  int64  
 4   date    60114 non-null  object 
 5   time    60114 non-null  object 
 6   tz      60114 non-null  int64  
 7   st      60114 non-null  object 
 8   stf     60114 non-null  int64  
 9   stn     60114 non-null  int64  
 10  mag     60114 non-null  int64  
 11  inj     60114 non-null  int64  
 12  fat     60114 non-null  int64  
 13  loss    60114 non-null  float64
 14  closs   60114 non-null  float64
 15  slat    60114 non-null  float64
 16  slon    60114 non-null  float64
 17  elat    60114 non-null  float64
 18  elon    60114 non-null  float64
 19  len     60114 non-null  float64
 20  wid     60114 non-null  int64  
 21  fc      60114 non-null  int64  
dty

#Preprocessing the data

In [42]:
def onehot_encoding(df,column):
    df=df.copy()
    dummies=pd.get_dummies(df[column],prefix=column)
    df=pd.concat([df,dummies],axis=1)
    df=df.drop(column,axis=1)
    return df

In [47]:
def preprocessing_input(df):
    df=df.copy()
    df=df.drop('date',axis=1)
    df=df.drop(['om','stn'],axis=1)
    #droping fc column
    df=df.drop('fc',axis=1)
    #extracting timeframe
    df['time']=pd.to_datetime(df['time'])
    df['hour']=df['time'].apply(lambda x:x.hour)
    df['minute']=df['time'].apply(lambda x:x.minute)
    df=df.drop('time',axis=1)
    #one hot encoding the nominal features
    for column in ['tz','st','stf']:
        df=onehot_encoding(df,column=column)
    y=df['mag']
    x=df.drop('mag',axis=1)
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,shuffle=True,random_state=1)
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),index=x_train.index,columns=x_train.columns)
    x_test=pd.DataFrame(scaler.transform(x_test),index=x_test.index,columns=x_test.columns)
    
    return x_train,x_test,y_train,y_test

In [48]:
x_train,x_test,y_train,y_test=preprocessing_input(df)
x_train

,yr,mo,dy,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,hour,minute,tz_0,tz_3,tz_6,tz_9,st_AK,st_AL,st_AR,st_AZ,st_CA,st_CO,st_CT,st_DC,st_DE,st_FL,st_GA,st_HI,st_IA,st_ID,st_IL,st_IN,st_KS,st_KY,st_LA,st_MA,st_MD,st_ME,st_MI,st_MN,st_MO,st_MS,st_MT,st_NC,st_ND,st_NE,st_NH,st_NJ,st_NM,st_NV,st_NY,st_OH,st_OK,st_OR,st_PA,st_PR,st_RI,st_SC,st_SD,st_TN,st_TX,st_UT,st_VA,st_VT,st_WA,st_WI,st_WV,st_WY,stf_1,stf_2,stf_4,stf_5,stf_6,stf_8,stf_9,stf_10,stf_11,stf_12,stf_13,stf_15,stf_16,stf_17,stf_18,stf_19,stf_20,stf_21,stf_22,stf_23,stf_24,stf_25,stf_26,stf_27,stf_28,stf_29,stf_30,stf_31,stf_32,stf_33,stf_34,stf_35,stf_36,stf_37,stf_38,stf_39,stf_40,stf_41,stf_42,stf_44,stf_45,stf_46,stf_47,stf_48,stf_49,stf_50,stf_51,stf_53,stf_54,stf_55,stf_56,stf_72
46779,0.897770,2.103604,-0.443014,-0.077047,-0.067993,-0.117031,-0.01555,-1.273971,1.081172,0.511311,-0.682013,-0.387409,-0.247680,-0.847778,0.825620,-0.011942,0.025339,-0.021806,-0.004875,-0.006894,-0.186881,-0.172797,-0.061781,-0.085588,-0.187085,-0.039635,0.0,-0.033794,4.220244,-0.158562,-0.026711,-0.205311,-0.056733,-0.202212,-0.152552,-0.265457,-0.124078,-0.178901,-0.050491,-0.073972,-0.046297,-0.128449,-0.171112,-0.19153,-0.186745,-0.079609,-0.146319,-0.159896,-0.220557,-0.038722,-0.047819,-0.098949,-0.034835,-0.084311,-0.129686,-0.255680,-0.041973,-0.111859,-0.018243,-0.012899,-0.126814,-0.173525,-0.138858,-0.407920,-0.044724,-0.105478,-0.027587,-0.043917,-0.147417,-0.044187,-0.106737,-0.186881,-0.006894,-0.061781,-0.172797,-0.085588,-0.187085,-0.039635,-0.033794,0.0,4.220244,-0.158562,-0.026711,-0.056521,-0.202212,-0.152552,-0.205374,-0.265457,-0.124078,-0.178901,-0.046297,-0.073972,-0.050491,-0.128449,-0.171112,-0.186745,-0.19153,-0.079609,-0.220557,-0.034835,-0.038722,-0.047819,-0.098949,-0.084311,-0.146319,-0.159896,-0.129781,-0.255680,-0.041973,-0.111859,-0.012899,-0.126814,-0.173525,-0.138858,-0.407880,-0.044724,-0.027587,-0.105478,-0.043917,-0.044187,-0.147417,-0.106737,-0.018243
55445,1.292855,-0.405231,-0.557424,-0.077047,-0.067993,0.435713,-0.01555,0.730677,-0.181883,1.054821,-0.919738,-0.292728,1.528395,-0.031984,1.107806,-0.011942,0.025339,-0.021806,-0.004875,-0.006894,-0.186881,-0.172797,-0.061781,-0.085588,-0.187085,-0.039635,0.0,-0.033794,-0.236953,-0.158562,-0.026711,4.870654,-0.056733,-0.202212,-0.152552,-0.265457,-0.124078,-0.178901,-0.050491,-0.073972,-0.046297,-0.128449,-0.171112,-0.19153,-0.186745,-0.079609,-0.146319,-0.159896,-0.220557,-0.038722,-0.047819,-0.098949,-0.034835,-0.084311,-0.129686,-0.255680,-0.041973,-0.111859,-0.018243,-0.012899,-0.126814,-0.173525,-0.138858,-0.407920,-0.044724,-0.105478,-0.027587,-0.043917,-0.147417,-0.044187,-0.106737,-0.186881,-0.006894,-0.061781,-0.172797,-0.085588,-0.187085,-0.039635,-0.033794,0.0,-0.236953,-0.158562,-0.026711,-0.056521,-0.202212,-0.152552,4.869163,-0.265457,-0.124078,-0.178901,-0.046297,-0.073972,-0.050491,-0.128449,-0.171112,-0.186745,-0.19153,-0.079609,-0.220557,-0.034835,-0.038722,-0.047819,-0.098949,-0.084311,-0.146319,-0.159896,-0.129781,-0.255680,-0.041973,-0.111859,-0.012899,-0.126814,-0.173525,-0.138858,-0.407880,-0.044724,-0.027587,-0.105478,-0.043917,-0.044187,-0.147417,-0.106737,-0.018243
29711,0.164042,-0.823370,-1.587113,-0.077047,-0.067993,-0.117584,-0.01555,-0.513183,-0.573431,-1.111276,1.122409,-0.399245,-0.349170,0.375913,0.204813,-0.011942,0.025339,-0.021806,-0.004875,-0.006894,-0.186881,-0.172797,-0.061781,-0.085588,-0.187085,-0.039635,0.0,-0.033794,-0.236953,-0.158562,-0.026711,-0.205311,-0.056733,-0.202212,-0.152552,-0.265457,-0.124078,-0.178901,-0.050491,-0.073972,-0.046297,-0.128449,-0.171112,-0.19153,-0.186745,-0.079609,-0.146319,-0.159896,-0.220557,-0.038722,-0.047819,-0.098949,-0.034835,-0.084311,-0.129686,3.911145,-0.041973,-0.111859,-0.018243,-0.012899,-0.126814,-0.173525,-0.138858,-0.407920,-0.044724,-0.105478,-0.027587,-0.043917,-0.147417,-0.044187,-0.106737,-0.186881,-0.006894,-0.061781,-0.172797,-0.085588,-0.187085,-0.039635,-0.033794,0.0,-0.236953,-0.158562,-0.026711,-

#Training Data

In [52]:
models={'Linear Regression':LinearRegression(),
        'Ridge(L2) Regression':Ridge(),
       'Lasso(L2) Regression':Lasso()}
for name,model in models.items():
    model.fit(x_train,y_train)
    print(name+' trained')

Linear Regression trained
Ridge(L2) Regression trained
Lasso(L2) Regression trained


Results

In [53]:
for name,model in models.items():
    print(name+'R^2 Score:{:5f}'.format(model.score(x_test,y_test)))

Linear RegressionR^2 Score:-442628400903886471168.000000
Ridge(L2) RegressionR^2 Score:0.413817
Lasso(L2) RegressionR^2 Score:-0.000045
